In [8]:
import pandas as pd
import numpy as np

In [22]:
class featureEng:

    def __init__(self, df):
        self.df = df

    #The definition of the function extract_changes
    def extract_changes(self):
        """ 
        Extracts the precentage, daily, and difference changes of high, low, volume, and close in the DataFrame. 

        Parameters: 
            self.df (DataFrame): DataFrame containing the stock price data.
        
        Parameter Constraints:
            Should have the columns high, low, volume, and close
        
        Returns:
            The modified DataFrame with the columns 'Daily Range', 'Volume Change', and 'Price Increase' added to previous DataFrame  
        """
        # Making a feature 'Daily Range' by substracting the elements low from high
        # and adding it after the 'Low' column
        self.df['Daily Range'] = self.df['High'] - self.df['Low']
        index_low = self.df.columns.get_loc('Low')
        self.df.insert(index_low + 1, 'Daily Range', self.df.pop('Daily Range'))
        
        # Making a feature 'Volume Change' by taking the percentage change in the volume
        # and adding it after 'Volume' column
        self.df['Volume Change'] = self.df['Volume'].pct_change()
        index_volume = self.df.columns.get_loc('Volume')
        self.df.insert(index_volume + 1, 'Volume Change', self.df.pop('Volume Change'))

        # Making a feature 'Close Price Increase' by having a 1 if the price increase or 
        # a 0 if it did not. Adding it after 'Close' column
        self.df['Close Price Increase'] = (self.df['Close'] > self.df['Close'].shift(1)).astype(int)
        index_close = self.df.columns.get_loc('Close')
        self.df.insert(index_close + 1, 'Close Price Increase', self.df.pop('Close Price Increase'))

        return self.df
    
    def volume_change(self):
        """ 
        Extracts the precentage change in the volume in the DataFrame. 

        Parameters: 
            self.df (DataFrame): DataFrame containing the stock price data.
        
        Parameter Constraints:
            Should have the column 'Volume'
        
        Returns:
            The modified DataFrame with the column 'Volume Change' added.  
        """
        # Making a feature 'Volume Change' by taking the percentage change in the volume
        # and adding it after 'Volume' column
        self.df['Volume Change'] = self.df['Volume'].pct_change()
        index_volume = self.df.columns.get_loc('Volume')
        self.df.insert(index_volume + 1, 'Volume Change', self.df.pop('Volume Change'))

        return self.df

    def close_price_increase(self):
        """ 
        Shows whether not the price increased in the close price of the DataFrame. 

        Parameters: 
        self.df (DataFrame): DataFrame containing the stock price data.
    
        Parameter Constraints:
        Should have the column 'Close'
    
        Returns:
        The modified DataFrame with the column 'Close Price Increase' added. 1 if the price increased and 0 if it did not.
        """
        # Making a feature 'Close Price Increase' by having a 1 if the price increase or 
        # a 0 if it did not. Adding it after 'Close' column
        self.df['Close Price Increase'] = (self.df['Close'] > self.df['Close'].shift(1)).astype(int)
        index_close = self.df.columns.get_loc('Close')
        self.df.insert(index_close + 1, 'Close Price Increase', self.df.pop('Close Price Increase'))

        return self.df
    
    def daily_range(self):
        """ 
        Extracts the 'Daily Range' change by substracting the elements of 'Low' from 'High' of the DataFrame. 

        Parameters: 
        self.df (DataFrame): DataFrame containing the stock price data.
    
        Parameter Constraints:
        Should have the columns 'High' and 'Low'
    
        Returns:
        The modified DataFrame with the column 'Daily Range' added.
        """
        # Making a feature 'Daily Range' by substracting the elements low from high
        # and adding it after the 'Low' column
        self.df['Daily Range'] = self.df['High'] - self.df['Low']
        index_low = self.df.columns.get_loc('Low')
        self.df.insert(index_low + 1, 'Daily Range', self.df.pop('Daily Range'))

        return self.df
    
    def calculate_macd(self, short_window=12, long_window=26, signal_window=9):
        """
        Calculate the Moving Average Convergence Divergence (MACD) indicator for a given DataFrame.

        Parameters:
            self.df (DataFrame): DataFrame containing the stock price data. It should have a 'Date' column and a 'Price' column.
            short_window (int): Number of days for the short-term moving average (default is 12).
            long_window (int): Number of days for the long-term moving average (default is 26).
            signal_window (int): Number of days for the signal line moving average (default is 9).

        Returns:
            The modified DataFrame containing the MACD, signal line, and MACD histogram.
        """
        # Calculate the short EMA
        self.df['shortEMA'] = self.df['Close'].ewm(span=short_window, adjust=False).mean()

        # Calulate the long EMA
        self.df['longEMA'] = self.df['Close'].ewm(span=long_window, adjust=False).mean()

        # Calculate the Mean Average Convergence Divergence
        self.df['MACD'] = self.df['shortEMA'] - self.df['longEMA']

        # Get the signal line
        self.df['signalLine'] = self.df['MACD'].ewm(span=signal_window, adjust=False).mean()

        # Calculate the MACD histogram
        self.df['MACD_histogram'] = self.df['MACD'] - self.df['signalLine']

        return self.df


### _The code blocks below is for testing the class and its functions_

In [28]:
df = pd.DataFrame({'Date' : ['2023-09-15', '2023-09-16', '2023-09-17'],
                   'High' : [21,22,23],
                   'Low' : [15,16,17],
                   'Volume' : [1200,2300,2400],
                   'Close' : [22,26,29]
                   })

df1 = featureEng(df).calculate_macd()
df1

,Date,High,Low,Volume,Close,shortEMA,longEMA,MACD,signalLine,MACD_histogram
0,2023-09-15,21,15,1200,22,22.000000,22.000000,0.000000,0.000000,0.000000
1,2023-09-16,22,16,2300,26,22.615385,22.296296,0.319088,0.063818,0.255271
2,2023-09-17,23,17,2400,29,23.597633,22.792867,0.804766,0.212007,0.592759
